In [111]:
import numpy as np
import pandas as pd
df2 = pd.read_csv('C:/Users/Mayank Gokhale/Desktop/movie.csv')

from sklearn.feature_extraction.text import TfidfVectorizer
#Define a TF_IDF Vectorizer Object. Remove all english stop words such as 'the','a'
tfidf = TfidfVectorizer(stop_words='english')
#replace NaN with and empty string
df2['overview'] = df2['overview'].fillna('')
#construct the required TF_IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['overview'])

from sklearn.metrics.pairwise import linear_kernel
cosine_sim= linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

#function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1],reverse =True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['title'].iloc[movie_indices]

In [112]:
%matplotlib inline
import datetime
import ast
import seaborn as sns
from scipy import stats
from ast import literal_eval

sns.set_style('whitegrid')
pd.set_option('display.max_colwidth', 50)

df = pd.read_csv('C:/Users/Mayank Gokhale/Desktop/movies_metadata.csv')
df = df.drop(['imdb_id'], axis=1)
df = df.drop('original_title', axis=1)
df['revenue'] = df['revenue'].replace(0, np.nan)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['budget'] = df['budget'].replace(0, np.nan)
df['return'] = df['revenue'] / df['budget']
f = df.drop('adult', axis=1)

df_fran = df[df['belongs_to_collection'].notnull()]
df_fran['belongs_to_collection'] = df_fran['belongs_to_collection'].apply(ast.literal_eval).apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)
df_fran = df_fran[df_fran['belongs_to_collection'].notnull()]

fran_sum = pd.DataFrame(df_fran.groupby('belongs_to_collection')['revenue'].sum().sort_values(ascending=False))
fran_sum.columns = ['Total']
fran_mean = pd.DataFrame(df_fran.groupby('belongs_to_collection')['revenue'].mean().sort_values(ascending=False))
fran_mean.columns = ['Average']
fran_count = pd.DataFrame(df_fran.groupby('belongs_to_collection')['revenue'].count().sort_values(ascending=False))
fran_count.columns = ['Number']
fran_pivot = pd.concat((fran_sum, fran_mean, fran_count), axis=1)

md = pd. read_csv('C:/Users/Mayank Gokhale/Desktop/movies_metadata.csv')
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.95)
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')


C:\Users\Mayank Gokhale\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-112-4cb9e5e0faef>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fran['belongs_to_collection'] = df_fran['belongs_to_collection'].apply(ast.literal_eval).apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)


In [113]:
def clean_numeric(x):
    try:
        return float(x)
    except:
        return np.nan
    
df['popularity'] = df['popularity'].apply(clean_numeric).astype('float')
df['vote_count'] = df['vote_count'].apply(clean_numeric).astype('float')
df['vote_average'] = df['vote_average'].apply(clean_numeric).astype('float')
df['vote_average'] = df['vote_average'].replace(0, np.nan)
df['runtime'] = df['runtime'].astype('float')
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
df['spoken_languages'] = df['spoken_languages'].fillna('[]').apply(ast.literal_eval).apply(lambda x: len(x) if isinstance(x, list) else np.nan)

In [114]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(250)

s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

<ipython-input-114-bafa4e19a695>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


In [ ]:
y = "yes"
while y=="yes":
    print("\t\t\tMOVIE RECOMMENDATION SYSTEM")
    print("1. Item Based    2. IMDB Rating    3. Genre Based    4. Content Based Filering")
    print("\n\n1. Item Based system will allow you to filter movies according to their \n-budget\n-revenue\n-return\n-spoken languages\n-most runtime\n-least runtime\n-Total earning of a movie franchise\n-Average earning of a movie in a franchise\n-Total movies in a franchise")
    print("\n2. It shows the top movies using the \n-IMDB weighted rating system \n-Popularity\n-Most Votes\n-Highest Base Rating")
    print("\n3. Genre based section will provide you with the best movies for the genre you have selected")
    print("\n4. You can find movies based on the movies you like !")
    z=int(input("\nEnter the number respective to the Recommendation you want: "))
    if(z==1):
        x=input('''"Enter the keyword if you want to access the following option:
                -Total : Total earnings of movies in the franchise
                -Average : Average earning of a movie in a franchise
                -Number : Total movies in a franchise
                -budget : The movies with the highest budget
                -revenue : The movies with the highest revenue
                -return : The movies with the highest return
                -languages : The movies which have the most number of lanugages spoken
                -most runtime : Movies with the most runtime
                -least runtime : Movies with the least runtime \n"''')
        if(x=="Total" or x=="Average" or x=="Number"):
            print(fran_pivot.sort_values(x, ascending=False).head(10))  #total average number
        elif(x=="budget" or x=="revenue" or x=="return"):
            print(df[df['budget'].notnull()][['title',x, 'year']].sort_values(x, ascending=False).head(10))
        #budget revenue return
        elif(x=="languages"):
            print(df[df['spoken_languages'] >= 9][['title', 'year', 'spoken_languages']].sort_values('spoken_languages', ascending=False))
        elif(x=="least runtime"):
            print(df[df['runtime'] > 0][['runtime', 'title', 'year']].sort_values('runtime').head(10))
        elif(x=="most runtime"):
            print(df[df['runtime'] > 0][['runtime', 'title', 'year']].sort_values('runtime', ascending=False).head(10))
        #most spoken languages, most runtime,least runtime
        
        
    elif(z==2):  
        x = input('''"Enter the keyword if you want to acccess the following option:
                  - IMDB : IMDB weighted rating
                  - popularity : Most popular movies
                  - vote_count : Most voted movies
                  - vote_average : The movies with the highest average rating
                  "''')
        if(x=="IMDB"):
            print(qualified.head(10))
        elif(x==vote_count or x== vote_average or x==popularity):
            print(df[df['vote_count'] > 2000][['title', x,'year' ]].sort_values(x, ascending=False).head(10))
        #popularity vote_count vote_average
        
    elif(z==3):
        print('''The following genres are availabe : 
        - Romance 
        - Comedy 
        - Action 
        - Western 
        - Fantasy 
        - Adventure 
        - Family 
        - Crime 
        - Thriller 
        - Horror 
        - Animation
        - History 
        - Science Fiction 
        - Mystery 
        - War\n''')
        x=input("Enter the Genre of the movie you want to watch :")
        print(build_chart(x).head(10))
         #Romance Comedy Action Western Fantasy Adventure Family Crime Thriller Horror Animation
         #History Science Fiction Mystery War   
        
        
    elif(z==4):
        print('\nEnter the movie name: ')
        x = input()
        print(get_recommendations(x))
    
    
    y=input("\nEnter 'yes' to go back to the menu or press any other key to exit\n")

			MOVIE RECOMMENDATION SYSTEM
1. Item Based    2. IMDB Rating    3. Genre Based    4. Content Based Filering


1. Item Based system will allow you to filter movies according to their 
-budget
-revenue
-return
-spoken languages
-most runtime
-least runtime
-Total earning of a movie franchise
-Average earning of a movie in a franchise
-Total movies in a franchise

2. It shows the top movies using the 
-IMDB weighted rating system 
-Popularity
-Most Votes
-Highest Base Rating

3. Genre based section will provide you with the best movies for the genre you have selected

4. You can find movies based on the movies you like !

Enter the number respective to the Recommendation you want: 3
The following genres are availabe : 
        - Romance 
        - Comedy 
        - Action 
        - Western 
        - Fantasy 
        - Adventure 
        - Family 
        - Crime 
        - Thriller 
        - Horror 
        - Animation
        - History 
        - Science Fiction 
        - Mystery 
